In [1]:
import requests
import folium
import json
from folium.plugins import MarkerCluster
import pandas as pd

# GeoJSON 파일 다운로드
url = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'
response = requests.get(url)
geo_data = response.json()

In [2]:
# GeoJSON 데이터를 데이터프레임으로 변환
data = []
for feature in geo_data['features']:
    district_name = feature['properties']['name']
    coordinates = feature['geometry']['coordinates'][0]
    for coord in coordinates:
        data.append([district_name, coord[1], coord[0]])  # GeoJSON의 좌표는 [경도, 위도] 형식이므로 순서 변경

# 데이터프레임 생성
df = pd.DataFrame(data, columns=['구 이름', '위도', '경도'])

# 서울 지도 생성
df_m = folium.Map(location=[37.5665, 126.9780], zoom_start=12)

# 마커 클러스터 생성
marker_cluster = MarkerCluster().add_to(df_m)

In [3]:
# 각 구별로 다른 색상 지정
colors = {
    '종로구': 'purple',
    '중구': 'blue',
    '용산구': 'green',
    '성동구': 'red',
    '광진구': 'orange',
    '동대문구': 'pink',
    '중랑구': 'yellow',
    '성북구': 'lightblue',
    '강북구': 'lightgreen',
    '도봉구': 'lightred',
    '노원구': 'lightorange',
    '은평구': 'lightpink',
    '서대문구': 'lightyellow',
    '마포구': 'purple',
    '양천구': 'blue',
    '강서구': 'green',
    '구로구': 'red',
    '금천구': 'orange',
    '영등포구': 'pink',
    '동작구': 'yellow',
    '관악구': 'lightblue',
    '서초구': 'lightgreen',
    '강남구': 'lightred',
    '송파구': 'lightorange',
    '강동구': 'lightpink'
}

In [4]:
# 구별 폴리곤 및 마커 추가
for district in df['구 이름'].unique():
    district_data = df[df['구 이름'] == district]
    coordinates = district_data[['위도', '경도']].values.tolist()

    # 경계선 폴리곤 만들기
    folium.Polygon(
        locations=coordinates,
        color='black',
        weight=1,
        fill=True,
        fill_color=colors.get(district, 'gray'),
        fill_opacity=0.6,
        popup=district
    ).add_to(df_m)

    # 중심 좌표 계산 (위도와 경도의 평균값 사용)
    center_lat = district_data['위도'].mean()
    center_lon = district_data['경도'].mean()

    # 마커 클러스터에 마커 추가
    folium.Marker(
        location=[center_lat, center_lon],
        popup=district,
        icon=folium.Icon(color='blue', icon='bolt')
    ).add_to(marker_cluster)

# 지도 저장
df_m.save('seoul_folium_color_cluster.html')

# 지도 출력
df_m